In [12]:
import sys
import os
import numpy as np

In [15]:
sys.path.insert(0, os.path.expanduser('~/git/refl1d_analysis/notebooks'))
import plot_utils

In [58]:
import refnx
import refl1d
from refl1d.names import *

from refnx.reflect._creflect import abeles
from refl1d.abeles import refl
from refl1d.reflectivity import reflectivity

In [6]:
def magsq(z):
    return z.real**2 + z.imag**2

In [43]:
q = np.linspace(0.008, 0.05, 500)

In [87]:
if False:
    depth = [0, 1200, 0]
    rho =   [2.07, 4.66, 6.36]
    irho =  [0, 0.016, 0]
    refl1d_sigma = [10, 3]
    refnx_sigma = [np.nan, 10, 3]
else:
    depth = [0,     4.94,  60,  850,   10, 0]
    rho =   [2.067, 3.025, 2.8, 4.264, 4,  6.05]
    irho =  [0,     0.,    0,   0.1,   0,  0]
    refl1d_sigma = [2.0, 1.6, 35, 1.017, 4.963]
    refnx_sigma = [np.nan, 2.0, 1.6, 33, 1.017, 4.963]

w = np.c_[depth, rho, irho, refnx_sigma]

r_nx = abeles(q, w)

r_1d_abeles = magsq(refl(q/2.0, depth, rho, irho=irho, sigma=refl1d_sigma))

r_1d = reflectivity(q/2.0, depth, rho, irho=irho, sigma=refl1d_sigma)

In [88]:
plot_utils.plot1d([[q, r_nx], [q, r_1d_abeles], [q, r_1d]],
                  data_names=['refnx', 'abeles', 'refl1d'],
                  y_log=True, x_log=True)

## For reference, this is closer to code I would use

In [89]:
def make_gold_experiment(q_min=0.008, q_max=0.05, npoints=500, steps=False, gold_thick=850, gold_rough=35):
    q = np.logspace(np.log10(q_min), np.log10(q_max), npoints)
    dq = 0.0*q
    r = np.zeros_like(q)
    probe = QProbe(q, dq, data=(r, r))

    # Materials ####################################################################
    Si = SLD(name='Si', rho=2.067, irho=0.0)
    d2o = SLD(name='Electrolyte', rho=6.05, irho=0.0)
    SiOx = SLD(name='SiOx', rho=3.025, irho=0.0)
    Cr = SLD(name='Cr', rho=2.8, irho=0.0)
    gold = SLD(name='Au', rho=4.264, irho=0.1)
    SAM = SLD(name='SAM', rho=4.0, irho=0.0)


    # Film definition ##############################################################
    sample = (  d2o(0, 4.963) | SAM(10.0, 1.017) | gold(gold_thick, gold_rough) | Cr(60, 1.6) | SiOx(4.94, 2.0) | Si )
    probe.intensity=Parameter(value=1.0,name='normalization')
    probe.background=Parameter(value=0,name='background')

    ################################################################################

    expt = Experiment(probe=probe, sample=sample, step_interfaces=steps, dz=2)
    return expt

In [90]:
expt = make_gold_experiment(steps=False)
r_0 = expt.reflectivity()

plot_utils.plot1d([[q, r_nx],
                   [q, r_1d_abeles],
                  [r_0[0], r_0[1]]],
                 data_names =  ['refnx', 'refl1d abeles', 'refl1d'],
                 y_log = True, x_log=True)